In [2]:
import pandas as pd
import numpy as np

In [3]:
!ls

48914275_outbound_dnn_run_2023-08-24 11:55:08.408130
48914302_outbound_dnn_run_2023-08-24 12:52:57.102379
48914452_outbound_stdnn_run_2023-08-24 16:40:00.282208
48914463_outbound_stdnn_run_2023-08-24 17:35:59.325309
48950875_outbound_stdnn_run_2023-08-28 10:49:53.851112
best
inference_48949457.txt
inference_48949458.txt
inference.py
__init__.py
models.py
_output
__pycache__
runs
soft_training_labels.parq
st_training_predictions.parq
student_learned_training_labels.parq
student_teacher.py
st_val_predictions.parq
submit_dnn.sh
submit_gbrfc.sh
submit_inference.sh
submit_rfc.sh
submit_st.sh
submit_tuner.sh
teacher_soft_training_labels.parq
train.py
tuner
Untitled.ipynb


In [43]:
teacher_pred = pd.read_parquet('teacher_soft_training_labels.parq')

In [44]:
teacher_pred

,label_1
0,0.021041
1,0.021554
2,0.354701
3,0.106668
4,0.009926
...,...
13194566,0.986173
13194567,0.985376
13194568,0.988153
13194569,0.987431


In [45]:
student_pred = pd.read_parquet('student_learned_training_labels.parq')

In [46]:
student_pred

,model_train_pred
0,0.011274
1,0.024941
2,0.408227
3,0.129211
4,0.013278
...,...
13194566,0.955553
13194567,0.960881
13194568,0.958336
13194569,0.965071


In [47]:
results = pd.concat([teacher_pred, student_pred], axis=1)

In [48]:
results

,label_1,model_train_pred
0,0.021041,0.011274
1,0.021554,0.024941
2,0.354701,0.408227
3,0.106668,0.129211
4,0.009926,0.013278
...,...,...
13194566,0.986173,0.955553
13194567,0.985376,0.960881
13194568,0.988153,0.958336
13194569,0.987431,0.965071


In [50]:
t_pred = results['label_1'].to_numpy()
s_pred = results['model_train_pred'].to_numpy()

diff = t_pred - s_pred
results['diff'] = pd.Series(data=diff)

In [51]:
results

,label_1,model_train_pred,diff
0,0.021041,0.011274,0.009767
1,0.021554,0.024941,-0.003387
2,0.354701,0.408227,-0.053526
3,0.106668,0.129211,-0.022543
4,0.009926,0.013278,-0.003352
...,...,...,...
13194566,0.986173,0.955553,0.030620
13194567,0.985376,0.960881,0.024495
13194568,0.988153,0.958336,0.029817
13194569,0.987431,0.965071,0.022361


In [52]:
np.mean(np.abs(diff))

0.03586939091931898

In [53]:
np.max(np.abs(diff))

0.8621090113388767

In [54]:
sort = np.sort(np.abs(diff))

In [55]:
sort

array([7.42141059e-09, 7.42141059e-09, 7.42141059e-09, ...,
       8.52781294e-01, 8.52788685e-01, 8.62109011e-01])

In [56]:
(sort > 0.1).sum()/len(sort)

0.07972051535438325

In [57]:
(sort > 0.1).sum()

1051878

In [42]:
!ls

48914275_outbound_dnn_run_2023-08-24 11:55:08.408130
48914302_outbound_dnn_run_2023-08-24 12:52:57.102379
48914452_outbound_stdnn_run_2023-08-24 16:40:00.282208
48914463_outbound_stdnn_run_2023-08-24 17:35:59.325309
48950875_outbound_stdnn_run_2023-08-28 10:49:53.851112
48950927_outbound_gbrfc_run2023-08-28 12:29:02.161626
48950962_outbound_gbrfc_run2023-08-28 13:30:57.548611
48950964_outbound_gbrfc_run2023-08-28 13:33:37.306402
best
inference_48949457.txt
inference_48949458.txt
inference.py
__init__.py
models.py
_output
__pycache__
runs
student_learned_test_labels.parq
student_learned_training_labels.parq
student_learned_val_labels.parq
student_teacher.py
Student Teacher Validation.ipynb
submit_dnn.sh
submit_gbrfc.sh
submit_inference.sh
submit_rfc.sh
submit_st.sh
submit_tuner.sh
teacher_soft_test_labels.parq
teacher_soft_training_labels.parq
teacher_soft_validation_labels.parq
train.py
tuner


In [60]:
t_val = pd.read_parquet('teacher_soft_validation_labels.parq').to_numpy()
s_val = pd.read_parquet('student_learned_val_labels.parq').to_numpy()
t_test = pd.read_parquet('teacher_soft_test_labels.parq').to_numpy()
s_test = pd.read_parquet('student_learned_test_labels.parq').to_numpy()

In [69]:
val_diff = t_val - s_val
test_diff = t_test - s_test

In [71]:
print(val_diff)
print(test_diff)

[[-0.00187227]
 [ 0.01729717]
 [-0.0123517 ]
 ...
 [ 0.02040807]
 [ 0.02040807]
 [-0.21556187]]
[[-0.03966042]
 [-0.01110816]
 [-0.07029392]
 ...
 [-0.00394075]
 [ 0.01780371]
 [-0.08690896]]


In [63]:
np.mean(np.abs(val_diff))

0.029662033448562494

In [66]:
(val_diff > 0.1).sum()/len(val_diff)

0.006238913141663344

In [72]:
np.mean(np.abs(test_diff))

0.02642390960655843

In [73]:
(test_diff > 0.1).sum()/len(test_diff)

0.010604630462120238

In [74]:
np.round(s_test)

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [0.]], dtype=float32)

In [75]:
np.round(t_test)

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [0.]])

In [78]:
(np.round(s_test) == np.round(t_test)).sum()/len(t_test)

0.9795860303539905

In [80]:
(np.round(s_test) == 1).sum()

110610

In [81]:
(np.round(t_test) == 1).sum()

81294

In [82]:
(np.round(t_test > 1))

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float16)

In [83]:
results

,label_1,model_train_pred,diff
0,0.021041,0.011274,0.009767
1,0.021554,0.024941,-0.003387
2,0.354701,0.408227,-0.053526
3,0.106668,0.129211,-0.022543
4,0.009926,0.013278,-0.003352
...,...,...,...
13194566,0.986173,0.955553,0.030620
13194567,0.985376,0.960881,0.024495
13194568,0.988153,0.958336,0.029817
13194569,0.987431,0.965071,0.022361


In [84]:
results.to_numpy()

array([[ 0.02104096,  0.01127434,  0.00976662],
       [ 0.02155414,  0.02494113, -0.00338699],
       [ 0.35470119,  0.40822715, -0.05352596],
       ...,
       [ 0.98815275,  0.958336  ,  0.02981676],
       [ 0.9874313 ,  0.96507061,  0.02236069],
       [ 0.98541273,  0.96088898,  0.02452375]])

In [96]:
import tensorflow as tf
from tensorflow import keras


In [97]:
model_path='/smallwork/alexander.huang/models/2023-09-05/48958189_outbound_stdnn_run_2023-09-05 17:36:51.434726/checkpoint100-0.17.h5'

In [98]:
model = keras.models.load_model(model_path)

In [99]:
model.summary()

Model: "stdnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 268)]             0         
                                                                 
 hidden_1 (Dense)            (None, 512)               137728    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 hidden_2 (Dense)            (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 temperature (Lambda)        (None, 512)               0         
                                                                 
 output (Dense)              (None, 2)                 1026  

In [106]:
model2 = keras.models.Sequential()
layers = [l for l in model.layers]
l1 = layers[:5]
l2 = [layers[6]]
layers = l1 + l2
for layer in layers:
    model2.add(layer)
model2 = keras.Model(inputs=model2.input, outputs=model2.output)

In [107]:
model2.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 268)]             0         
                                                                 
 hidden_1 (Dense)            (None, 512)               137728    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 hidden_2 (Dense)            (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 output (Dense)              (None, 2)                 1026      
                                                                 
Total params: 401410 (1.53 MB)
Trainable params: 401410 (1